# 휴먼지능정보공학과 201810808 정민지

## 연습문제 Multiply7



앞서 Multiply7을 구현해 보았다.

이 컨트랙에 이벤트를 추가해서 구현해보자.

geth 또는 ganache 가운데 어느 것이나 선택해서 실행해도 좋다.

아래 주어진 미완성 코드를 구현해서 노드에서 이벤트를 띄워보자.



* 함수는 param4인자를 받아서 곱하기 7 연산을 한다. 8과 16의 연산을 실행하자. 그 결과는 56과 112가 출력될 것이다.

* 이벤트로 넘겨주는 매개변수 param1은 누가 이 함수를 호출했는지 (address), param2는 언제 호출되었는지 (timestamp), 마지막으로 param3는 연산결과를 출력하도록 한다.



* 미완성 코드
```
contract Multiply7Event {
    event Print(param1, param2, param3);
    function multiply(param4)
```



#### 단계 1. 컨트랙 개발

In [33]:
%%writefile src/Multiply7Event.sol
pragma solidity 0.5.1;
contract Multiply7Event {
    event Print(address param1,uint256 param2,uint param3);
    function multiply(uint param4) public{
        emit Print(msg.sender,now,param4*7);
    }
}

Overwriting src/Multiply7Event.sol


#### 단계 2. 컴파일

In [34]:
!solc --gas --abi --bin src/Multiply7Event.sol


======= src/Multiply7Event.sol:Multiply7Event =======
Gas estimation:
construction:
   105 + 56400 = 56505
external:
   multiply(uint256):	1854
Binary: 
608060405234801561001057600080fd5b5061011a806100206000396000f3fe6080604052600436106039576000357c010000000000000000000000000000000000000000000000000000000090048063c6888fa114603e575b600080fd5b348015604957600080fd5b50607360048036036020811015605e57600080fd5b81019080803590602001909291905050506075565b005b7f91da4985ab616136202f4e81fd2d8cac1eb12591132d609cece407f7c6fb9205334260078402604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001838152602001828152602001935050505060405180910390a15056fea165627a7a723058208e3f4dc7f2f9fdf9be377dd06070462d126d774009a418d98908c211a63560c40029
Contract JSON ABI 
[{"constant":false,"inputs":[{"name":"param4","type":"uint256"}],"name":"multiply","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"index

#### 단계 3. 컨트랙 배포

In [39]:
%%writefile src/Multiply7EventDeploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var _abiArray = [{"constant":false,"inputs":[{"name":"param4","type":"uint256"}],"name":"multiply","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"param1","type":"address"},{"indexed":false,"name":"param2","type":"uint256"},{"indexed":false,"name":"param3","type":"uint256"}],"name":"Print","type":"event"}];
var _bin = "0x" + "608060405234801561001057600080fd5b5061011a806100206000396000f3fe6080604052600436106039576000357c010000000000000000000000000000000000000000000000000000000090048063c6888fa114603e575b600080fd5b348015604957600080fd5b50607360048036036020811015605e57600080fd5b81019080803590602001909291905050506075565b005b7f91da4985ab616136202f4e81fd2d8cac1eb12591132d609cece407f7c6fb9205334260078402604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001838152602001828152602001935050505060405180910390a15056fea165627a7a723058208e3f4dc7f2f9fdf9be377dd06070462d126d774009a418d98908c211a63560c40029";
async function deploy() {
    const accounts = await web3.eth.getAccounts();//값 반환 기다림
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/Multiply7EventDeploy.js


In [40]:
!node src/Multiply7EventDeploy.js

Deploying the contract from 0x008D3582CBB59Ba9B39be6A03C43528F90416f20
hash: 0x6c5bac66dc5697e95b12f03b1a223e1fa7545ef15b1b3063c879e0635d2d9946
---> The contract deployed to: 0x21129C25b10896f2824541Fe5cDc55B4fc5538C8


#### 단계 4. 사용

In [88]:
%%writefile src/Multiply7EventUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var _abiArray = [{"constant":false,"inputs":[{"name":"param4","type":"uint256"}],"name":"multiply","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"param1","type":"address"},{"indexed":false,"name":"param2","type":"uint256"},{"indexed":false,"name":"param3","type":"uint256"}],"name":"Print","type":"event"}];
var _test = new web3.eth.Contract(_abiArray, '0x21129C25b10896f2824541Fe5cDc55B4fc5538C8');
var event = _test.events.Print({fromBlock: 0}, function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    //console.log("Account: " + accounts[0]);
    const value = await _test.methods.multiply(8)
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
    console.log(value.events.Print.returnValues);
    console.log("\n-------------------------answer-------------------------");
    console.log("address:",value.events.Print.returnValues[0]);
    console.log("timestamp:",value.events.Print.returnValues[1]);
    console.log("result:",value.events.Print.returnValues[2]);
    console.log("--------------------------------------------------------\n");
    
    const value2 = await _test.methods.multiply(16)
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
    console.log(value2.events.Print.returnValues);
    console.log("\n-------------------------answer-------------------------");
    console.log("address:",value2.events.Print.returnValues[0]);
    console.log("timestamp:",value2.events.Print.returnValues[1]);
    console.log("result:",value2.events.Print.returnValues[2]);
    console.log("--------------------------------------------------------\n");
}
doIt()

Overwriting src/Multiply7EventUse.js


## 답

In [89]:
!node src/Multiply7EventUse.js

Result {
  '0': '0x008D3582CBB59Ba9B39be6A03C43528F90416f20',
  '1': '1591514514',
  '2': '56',
  param1: '0x008D3582CBB59Ba9B39be6A03C43528F90416f20',
  param2: '1591514514',
  param3: '56'
}

-------------------------answer-------------------------
address: 0x008D3582CBB59Ba9B39be6A03C43528F90416f20
timestamp: 1591514514
result: 56
--------------------------------------------------------

Result {
  '0': '0x008D3582CBB59Ba9B39be6A03C43528F90416f20',
  '1': '1591514514',
  '2': '112',
  param1: '0x008D3582CBB59Ba9B39be6A03C43528F90416f20',
  param2: '1591514514',
  param3: '112'
}

-------------------------answer-------------------------
address: 0x008D3582CBB59Ba9B39be6A03C43528F90416f20
timestamp: 1591514514
result: 112
--------------------------------------------------------

